In [1]:
# Play Atari PongNoFrameskip-v4 with DQN

In [2]:
import gym
import time
import argparse
import numpy as np

import torch

from lib import wrappers
from lib import dqn_model

import collections

In [3]:
DEFAULT_ENV_NAME = "PongNoFrameskip-v4"
FPS = 60

In [4]:
# parser = argparse.ArgumentParser()
# parser.add_argument("-m", "--model", required=True, help="Model file to load")
# parser.add_argument("-e", "--env", default=DEFAULT_ENV_NAME,
#                     help="Environment name to use, default=" + DEFAULT_ENV_NAME)
# parser.add_argument("-r", "--record", help="Directory to store video recording")
# parser.add_argument("--no-visualize", default=True, action='store_false', dest='visualize',
#                     help="Disable visualization of the game play")
# args = parser.parse_args()

VISUALIZE = True

env = wrappers.make_env(DEFAULT_ENV_NAME)
env = gym.wrappers.Monitor(env, "recording", force=True)
net = dqn_model.DQN(env.observation_space.shape, env.action_space.n)
net.load_state_dict(torch.load("dqn_pong-best.dat"))

state = env.reset()
total_reward = 0.0
action_counter = collections.Counter()

while True:
    start_ts = time.time()
    if VISUALIZE:
        env.render()
    state_v = torch.tensor(np.array([state], copy=False))
    q_vals = net(state_v).data.numpy()[0]
    action = np.argmax(q_vals)
    action_counter[action] += 1
    state, reward, done, _ = env.step(action)
    total_reward += reward
    if done:
        break
    if VISUALIZE:
        delta = 1/FPS - (time.time() - start_ts)
        if delta > 0:
            time.sleep(delta)
print("Total reward: %.2f" % total_reward)
print("Action counts:", action_counter)
env.close()

Total reward: 21.00
Action counts: Counter({3: 677, 5: 354, 4: 231, 2: 178, 1: 99, 0: 91})
